# kedro Test

    codiet_kedro_project/
    ├── conf/
    │   └── base/
    │       ├── catalog.yml
    │       └── parameters.yml
    ├── data/
    │   ├── REDcap_data_combined_nov2024.xlsx
    │   ├── DBN_test_data.csv
    │   └── CoDiet_Sociodemographics_abbre.csv
    ├── src/
    │   └── codiet/
    │       ├── pipelines/
    │       │   └── main/
    │       │       ├── data_cleaning.py
    │       │       ├── data_cleaning_nodes.py
    │       │       ├── feature_selection.py
    │       │       ├── pipeline.py
    │       |       └── pipeline_registry.py
    │       ├── run.py
    │       └── __init__.py
    ├── demo/
    │   └── notebook.py
    ├── tests/
    │   └── test_nodes.py
    └── README.md


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Causality/Methods/DNN/codiet_kedro_project/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 在Colab中测试Kedro项目
# !pip install kedro==0.18.12 pandas==2.0.3 numpy==1.24.3 scikit-learn==1.3.0 torch==2.0.1 seaborn==0.12.2 matplotlib==3.7.2 openpyxl==3.1.2
!pip install scikit-learn==1.3.0 torch==2.0.1 seaborn==0.12.2 matplotlib==3.7.2 openpyxl==3.1.2
!pip install wheel setuptools cython
!pip install "numpy>=1.26,<2.0" "pandas>=2.1,<2.2" "kedro==0.18.12"


In [ ]:
import numpy as np
import pandas as pd
from src.codiet.pipelines.main import pipeline, data_cleaning_nodes, feature_selection



## 用 KedroContext 加载项目配置并尝试读取 DataCatalog 的数据集

In [ ]:
from kedro.framework.context import KedroContext
from kedro.framework.hooks import _create_hook_manager
from pathlib import Path
from kedro.config import ConfigLoader

project_path = Path.cwd()
package_name = "codiet"
conf_path = project_path / "conf"
env = "base"
config_loader = ConfigLoader(conf_source=str(conf_path), env=env)


try:
    context = KedroContext(package_name=package_name, project_path=project_path, config_loader=config_loader, hook_manager=_create_hook_manager())
    print("KedroContext created OK")
except Exception as e:
    print("KedroContext creation failed:", e)
    raise

try:
    catalog = context.catalog
    print("Catalog datasets:", list(catalog.list()))
except Exception as e:
    print("Catalog load error:", e)

catalog = context.catalog
for key in list(catalog.list())[:5]:
    try:
        ds = catalog.load(key)
        print(f"Loaded {key} -> type: {type(ds)}, shape(if dataframe): {getattr(ds,'shape',None)}")
    except Exception as e:
        print(f"Failed to load {key}: {e}")


KedroContext created OK


[10/22/25 14:23:15] WARNING  /usr/local/lib/python3.11/dist-packages/kedro/config/common.py:216:    ]8;id=120423;file:///usr/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=284504;file:///usr/lib/python3.11/warnings.py#110\110]8;;\
                             UserWarning: Duplicate environment detected! Skipping re-loading from                 
                             configuration path: /content/drive/MyDrive/Colab                                      
                             Notebooks/Causality/Methods/DNN/codiet_kedro_project/conf/base                        
                               warn(                                                                               
                                                                                                                   

Catalog datasets: ['sociodemographics_data', 'exdbn_data', 'abbreviation_data', 'cleaned_data', 'codiet_dic_data', 'correlation_heatmap', 'parameters', 'params:feature_selection', 'params:feature_selection.threshold', 'params:feature_selection.method', 'params:data_cleaning', 'params:data_cleaning.remove_nulls', 'params:data_cleaning.fill_method']


                    INFO     Loading data from 'sociodemographics_data' (ExcelDataSet)...       ]8;id=709435;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=113252;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

Loaded sociodemographics_data -> type: <class 'pandas.core.frame.DataFrame'>, shape(if dataframe): (19, 94)


...

Loaded abbreviation_data -> type: <class 'pandas.core.frame.DataFrame'>, shape(if dataframe): (334, 3)


                    INFO     Loading data from 'cleaned_data' (CSVDataSet)...                   ]8;id=77313;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=23094;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

Loaded cleaned_data -> type: <class 'pandas.core.frame.DataFrame'>, shape(if dataframe): (304, 1)


                    INFO     Loading data from 'codiet_dic_data' (CSVDataSet)...                ]8;id=849434;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=869926;file:///usr/local/lib/python3.11/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

Loaded codiet_dic_data -> type: <class 'pandas.core.frame.DataFrame'>, shape(if dataframe): (440, 7)


## 验证 Kedro CLI 能读取配置、运行 pipeline、执行所有节点并写入 outputs（catalog 指定的位置）

In [ ]:
import subprocess
import sys

cmd = [
    sys.executable,
    "-W", "default:Kedro is not yet fully compatible",
    "-m", "kedro", "run",
    "--"
]

# 捕获输出
result = subprocess.run(cmd, capture_output=True, text=True)

print("stdout:")
print(result.stdout)
print("\nstderr:")
print(result.stderr)

print("\n返回码:", result.returncode)


stdout:
[10/22/25 14:30:47] INFO     Kedro project codiet_kedro_project   session.py:364
                    WARNING  /usr/local/lib/python3.11/dist-pack warnings.py:110
                             ages/kedro/framework/session/sessio                
                             n.py:266: FutureWarning:                           
                             ConfigLoader will be deprecated in                 
                             Kedro 0.19. Please use the                         
                             OmegaConfigLoader instead. To                      
                             consult the documentation for                      
                             OmegaConfigLoader, see here:                       
                             https://docs.kedro.org/en/stable/co                
                             nfiguration/advanced_configuration.                
                             html#omegaconfigloader                             
                    

## 写入pyproject.toml,如果没有

In [ ]:
from pathlib import Path

p = Path("pyproject.toml")
if not p.exists():
    p.write_text(
        '[tool.kedro]\n'
        'package_name = "codiet"\n'
        'project_name = "codiet_kedro_project"\n'
        'kedro_init_version = "0.18.12"\n'
        'source_dir = "src"\n',
        encoding='utf-8'
    )
print("pyproject.toml exists:", p.exists())


pyproject.toml exists: True


In [ ]:
from kedro.framework.cli import main
import subprocess
import sys

def run_kedro():
    cmd = [
        sys.executable, "-W",
        "default:Kedro is not yet fully compatible",
        "-m", "kedro", "run", "--",  # <-- 注意这里加 "--" 告诉 Click 停止解析后续 Jupyter 参数
    ]
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    run_kedro()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:14                                                                             │
│                                                                                                  │
│   11 │   subprocess.run(cmd, check=True)                                                         │
│   12                                                                                             │
│   13 if __name__ == "__main__":                                                                  │
│ ❱ 14 │   run_kedro()                                                                             │
│   15                                                                                             │
│                                                                                                  │
│ in run_kedro:11                                                                                  │
│                                                                                                  │
│    8 │   │   "default:Kedro is not yet fully compatible",                                        │
│    9 │   │   "-m", "kedro", "run", "--",  # <-- 注意这里加 "--" 告诉 Click 停止解析后续 Jupyt    │
│   10 │   ]                                                                                       │
│ ❱ 11 │   subprocess.run(cmd, check=True)                                                         │
│   12                                                                                             │
│   13 if __name__ == "__main__":                                                                  │
│   14 │   run_kedro()                                                                             │
│                                                                                                  │
│ /usr/lib/python3.11/subprocess.py:571 in run                                                     │
│                                                                                                  │
│    568 │   │   │   raise                                                                         │
│    569 │   │   retcode = process.poll()                                                          │
│    570 │   │   if check and retcode:                                                             │
│ ❱  571 │   │   │   raise CalledProcessError(retcode, process.args,                               │
│    572 │   │   │   │   │   │   │   │   │    output=stdout, stderr=stderr)                        │
│    573 │   return CompletedProcess(process.args, retcode, stdout, stderr)                        │
│    574                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
CalledProcessError: Command '['/usr/bin/python3', '-W', 'default:Kedro is not yet fully compatible', '-m', 'kedro',
'run', '--']' returned non-zero exit status 1.

In [ ]:
from pathlib import Path

package_name = "codiet"
src_dir = Path("src")
package_dir = src_dir / package_name

# 创建目录
package_dir.mkdir(parents=True, exist_ok=True)

# 创建 __init__.py
(package_dir / "__init__.py").touch(exist_ok=True)

# 创建空的 settings.py
settings_file = package_dir / "settings.py"
if not settings_file.exists():
    settings_file.write_text("# Kedro settings placeholder\n", encoding="utf-8")

print("项目结构已创建/检查完成：")
print(list(src_dir.rglob("*")))


项目结构已创建/检查完成：
[PosixPath('src/.DS_Store'), PosixPath('src/codiet'), PosixPath('src/codiet/pipelines'), PosixPath('src/codiet/__init__.py'), PosixPath('src/codiet/.DS_Store'), PosixPath('src/codiet/run.py'), PosixPath('src/codiet/__pycache__'), PosixPath('src/codiet/settings.py'), PosixPath('src/codiet/pipeline_registry.py'), PosixPath('src/codiet/pipelines/__init__.py'), PosixPath('src/codiet/pipelines/main'), PosixPath('src/codiet/pipelines/.DS_Store'), PosixPath('src/codiet/pipelines/__pycache__'), PosixPath('src/codiet/pipelines/main/__init__.py'), PosixPath('src/codiet/pipelines/main/.DS_Store'), PosixPath('src/codiet/pipelines/main/__pycache__'), PosixPath('src/codiet/pipelines/main/data_cleaning_nodes.py'), PosixPath('src/codiet/pipelines/main/feature_selection.py'), PosixPath('src/codiet/pipelines/main/pipeline.py'), PosixPath('src/codiet/pipelines/main/__pycache__/pipeline.cpython-311.pyc'), PosixPath('src/codiet/pipelines/main/__pycache__/data_cleaning_nodes.cpython-311.pyc'),